In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

class OptimalInsulinRecommender:
    def __init__(self, sequence_length=10, hidden_dim=64):
        self.sequence_length = sequence_length
        self.hidden_dim = hidden_dim
        self.scaler_X = MinMaxScaler()
        self.scaler_basal = MinMaxScaler()
        self.scaler_bolus = MinMaxScaler()
        self.model = None

    def prepare_features(self, df):
        """Enhanced feature engineering based on what worked best"""
        # Time features
        df['hour'] = df['time'].dt.hour
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

        # Glucose dynamics (proven effective)
        df['glucose_trend'] = df['glucose'].diff().fillna(0)
        df['glucose_acceleration'] = df['glucose_trend'].diff().fillna(0)
        df['glucose_above_180'] = (df['glucose'] > 180).astype(int)  # More relevant threshold

        # Meal and insulin context
        df['carbs_recent'] = df['carb_input'].rolling(window=4, min_periods=1).sum().fillna(0)
        df['time_since_bolus'] = df['bolus_volume_delivered'].replace(0, np.nan).ffill().fillna(180)

        # Activity context
        df['activity_level'] = (df['steps'] > 0).astype(int) + (df['heart_rate'] > 80).astype(int)

        # Proven effective features from previous success
        features = [
            'glucose', 'glucose_trend', 'glucose_acceleration', 'glucose_above_180',
            'carb_input', 'carbs_recent', 'time_since_bolus',
            'activity_level', 'hour_sin', 'hour_cos'
        ]

        return df, features

    def create_sequences(self, X, y):
        """Create sequences with proven 10-timestep window"""
        Xs, ys = [], []
        for i in range(len(X) - self.sequence_length):
            Xs.append(X[i:i+self.sequence_length])
            ys.append(y[i+self.sequence_length])
        return np.array(Xs), np.array(ys)

    def create_perfectly_balanced_dataset(self, X, y):
        """Create 50-50 balanced dataset (proven effective)"""
        bolus_indices = np.where(y[:, 2] == 1)[0]
        non_bolus_indices = np.where(y[:, 2] == 0)[0]

        # Take equal numbers for perfect balance
        np.random.seed(42)
        non_bolus_sample = np.random.choice(non_bolus_indices, len(bolus_indices), replace=False)

        balanced_indices = np.concatenate([bolus_indices, non_bolus_sample])
        np.random.shuffle(balanced_indices)

        return X[balanced_indices], y[balanced_indices]

    def build_optimal_model(self, input_dim):
        """Build the architecture that gave 75% recall"""
        class OptimalInsulinGRU(nn.Module):
            def __init__(self, input_dim, hidden_dim):
                super().__init__()
                # GRU with optimal hidden size
                self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, dropout=0.2)

                # Basal head - keep it simple
                self.basal_head = nn.Linear(hidden_dim, 1)

                # Bolus system - proven effective structure
                self.bolus_detector = nn.Sequential(
                    nn.Linear(hidden_dim, 32),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(32, 1),
                    nn.Sigmoid()  # Probability output
                )

                self.bolus_amount = nn.Sequential(
                    nn.Linear(hidden_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, 1),
                    nn.ReLU()  # Bolus can't be negative
                )

            def forward(self, x):
                gru_out, _ = self.gru(x)
                last_hidden = gru_out[:, -1, :]

                basal = self.basal_head(last_hidden)
                bolus_prob = self.bolus_detector(last_hidden)
                bolus_amt = self.bolus_amount(last_hidden)

                return basal, bolus_amt, bolus_prob

        return OptimalInsulinGRU(input_dim, self.hidden_dim)

    def train_with_optimal_settings(self, df, epochs=100):
        """Training with settings that gave best results"""
        print(" Training Optimal Insulin Recommender...")

        # Prepare data with proven features
        df, features = self.prepare_features(df)
        targets = ['basal_rate', 'bolus_volume_delivered']

        # Handle missing data
        df[features + targets] = df[features + targets].ffill().fillna(0)

        # Create bolus indicator
        df['bolus_indicator'] = (df['bolus_volume_delivered'] > 0.1).astype(int)
        bolus_count = df['bolus_indicator'].sum()
        print(f" Dataset: {len(df)} records, {bolus_count} bolus events ({bolus_count/len(df):.1%})")

        # Scale features
        X_scaled = self.scaler_X.fit_transform(df[features])
        basal_scaled = self.scaler_basal.fit_transform(df[['basal_rate']])
        bolus_scaled = self.scaler_bolus.fit_transform(df[['bolus_volume_delivered']])
        y_scaled = np.column_stack([basal_scaled, bolus_scaled, df['bolus_indicator'].values])

        # Create sequences
        X_seq, y_seq = self.create_sequences(X_scaled, y_scaled)
        print(f" Created {len(X_seq)} sequences")

        # Create perfectly balanced dataset (PROVEN APPROACH)
        X_balanced, y_balanced = self.create_perfectly_balanced_dataset(X_seq, y_seq)
        print(f" Balanced dataset: {len(X_balanced)} sequences (50% bolus, 50% no-bolus)")

        # Split with proper shuffling
        X_train, X_test, y_train, y_test = train_test_split(
            X_balanced, y_balanced, test_size=0.2, random_state=42, shuffle=True
        )

        # Convert to tensors
        X_train = torch.FloatTensor(X_train)
        y_train = torch.FloatTensor(y_train)
        X_test = torch.FloatTensor(X_test)
        y_test = torch.FloatTensor(y_test)

        # Build optimal model
        self.model = self.build_optimal_model(len(features))

        # Optimal optimizer settings from best run
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001, weight_decay=1e-4)

        # Loss functions
        criterion_basal = nn.MSELoss()
        criterion_bolus = nn.MSELoss()
        criterion_detect = nn.BCELoss()

        # Training history
        train_history = []

        print(" Training progress:")
        for epoch in range(epochs):
            self.model.train()
            optimizer.zero_grad()

            basal_pred, bolus_amt_pred, bolus_prob_pred = self.model(X_train)

            # Calculate losses with optimal weighting
            basal_loss = criterion_basal(basal_pred, y_train[:, 0:1])
            bolus_loss = criterion_bolus(bolus_amt_pred, y_train[:, 1:2])
            detect_loss = criterion_detect(bolus_prob_pred, y_train[:, 2:3])

            # PROVEN WEIGHTING: Heavy focus on detection
            total_loss = basal_loss + bolus_loss + (detect_loss * 3.0)

            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            optimizer.step()

            # Track metrics every epoch
            with torch.no_grad():
                bolus_detected = (bolus_prob_pred > 0.5).float()
                recall = ((bolus_detected == 1) & (y_train[:, 2:3] == 1)).float().sum() / (y_train[:, 2:3] == 1).float().sum()
                accuracy = (bolus_detected == y_train[:, 2:3]).float().mean()

                train_history.append({
                    'epoch': epoch + 1,
                    'total_loss': total_loss.item(),
                    'recall': recall.item(),
                    'accuracy': accuracy.item()
                })

            if (epoch + 1) % 20 == 0:
                print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss.item():.4f} - "
                      f"Recall: {recall.item():.3f} - Acc: {accuracy.item():.3f}")

        return X_test, y_test, train_history

    def comprehensive_evaluate(self, X_test, y_test):
        """Comprehensive evaluation with clinical insights"""
        self.model.eval()
        with torch.no_grad():
            basal_pred, bolus_amt_pred, bolus_prob_pred = self.model(X_test)

        # Convert predictions
        bolus_detected = (bolus_prob_pred > 0.5).float().numpy().flatten()
        bolus_true = y_test[:, 2].numpy().astype(int)

        # Inverse scale to actual units
        basal_pred_actual = self.scaler_basal.inverse_transform(basal_pred.numpy())
        bolus_amt_pred_actual = self.scaler_bolus.inverse_transform(bolus_amt_pred.numpy())
        basal_true_actual = self.scaler_basal.inverse_transform(y_test[:, 0:1].numpy())
        bolus_true_actual = self.scaler_bolus.inverse_transform(y_test[:, 1:2].numpy())

        # Calculate key metrics
        recall = recall_score(bolus_true, bolus_detected, zero_division=0)
        precision = precision_score(bolus_true, bolus_detected, zero_division=0)
        basal_mae = mean_absolute_error(basal_true_actual, basal_pred_actual)

        # Bolus amount accuracy for correctly detected events
        correct_detections = (bolus_detected == 1) & (bolus_true == 1)
        if correct_detections.any():
            bolus_mae = mean_absolute_error(bolus_true_actual[correct_detections],
                                          bolus_amt_pred_actual[correct_detections])
            bolus_count = correct_detections.sum()
        else:
            bolus_mae = float('nan')
            bolus_count = 0

        # Detailed breakdown
        true_positives = ((bolus_detected == 1) & (bolus_true == 1)).sum()
        false_positives = ((bolus_detected == 1) & (bolus_true == 0)).sum()
        false_negatives = ((bolus_detected == 0) & (bolus_true == 1)).sum()
        true_negatives = ((bolus_detected == 0) & (bolus_true == 0)).sum()

        print(f"\n" + "="*60)
        print(" COMPREHENSIVE PERFORMANCE REPORT")
        print("="*60)

        print(f"\n BOLUS DETECTION PERFORMANCE:")
        print(f"   Recall:    {recall:.1%} ({true_positives}/{true_positives + false_negatives} events detected)")
        print(f"   Precision: {precision:.1%} ({true_positives}/{true_positives + false_positives} correct)")
        print(f"   Accuracy:  {(true_positives + true_negatives) / len(bolus_true):.1%}")

        print(f"\n INSULIN DOSING ACCURACY:")
        print(f"   Basal Rate MAE:  {basal_mae:.4f} units")
        if bolus_count > 0:
            print(f"   Bolus Amount MAE: {bolus_mae:.3f} units ({bolus_count} events)")
        else:
            print(f"   Bolus Amount: No correctly detected events to evaluate")

        print(f"\n DETECTION BREAKDOWN:")
        print(f"   True Positives:  {true_positives:2d} (correct bolus predictions)")
        print(f"   False Positives: {false_positives:2d} (false alarms)")
        print(f"   False Negatives: {false_negatives:2d} (missed bolus events)")
        print(f"   True Negatives:  {true_negatives:2d} (correct no-bolus predictions)")

        # Clinical assessment
        print(f"\n CLINICAL ASSESSMENT:")
        if recall >= 0.7:
            print("    EXCELLENT - Highly reliable bolus detection")
        elif recall >= 0.5:
            print("    GOOD - Clinically useful detection rate")
        elif recall >= 0.3:
            print("     MODERATE - Limited clinical utility")
        else:
            print("    POOR - Needs significant improvement")

        if bolus_count > 0 and bolus_mae < 2.0:
            print("    GOOD - Reasonable bolus amount accuracy")
        elif bolus_count > 0:
            print("     MODERATE - Bolus amount needs improvement")

        return {
            'recall': recall,
            'precision': precision,
            'bolus_mae': bolus_mae,
            'basal_mae': basal_mae,
            'true_positives': true_positives,
            'false_positives': false_positives,
            'false_negatives': false_negatives
        }

# Usage with optimal settings
def main():
    # Load and prepare data
    df = pd.read_csv("HUPA0001P.csv", sep=';')
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').reset_index(drop=True)

    print(f" Data range: {df['time'].min()} to {df['time'].max()}")

    # Initialize and train optimal recommender
    recommender = OptimalInsulinRecommender(sequence_length=10, hidden_dim=64)

    # Train with optimal settings
    X_test, y_test, history = recommender.train_with_optimal_settings(df, epochs=100)

    # Comprehensive evaluation
    metrics = recommender.comprehensive_evaluate(X_test, y_test)

    print(f"\n" + "="*50)
    print(" OPTIMAL INSULIN RECOMMENDER READY!")
    print("="*50)

if __name__ == "__main__":
    main()

 Data range: 2018-06-13 18:40:00 to 2018-06-27 23:55:00
 Training Optimal Insulin Recommender...
 Dataset: 4096 records, 76 bolus events (1.9%)
 Created 4086 sequences
 Balanced dataset: 152 sequences (50% bolus, 50% no-bolus)
 Training progress:
Epoch 20/100 - Loss: 2.2232 - Recall: 0.984 - Acc: 0.570
Epoch 40/100 - Loss: 2.1504 - Recall: 0.719 - Acc: 0.595
Epoch 60/100 - Loss: 2.0641 - Recall: 0.625 - Acc: 0.587
Epoch 80/100 - Loss: 1.9213 - Recall: 0.688 - Acc: 0.669
Epoch 100/100 - Loss: 1.8644 - Recall: 0.625 - Acc: 0.653

 COMPREHENSIVE PERFORMANCE REPORT

 BOLUS DETECTION PERFORMANCE:
   Recall:    66.7% (8/12 events detected)
   Precision: 61.5% (8/13 correct)
   Accuracy:  71.0%

 INSULIN DOSING ACCURACY:
   Basal Rate MAE:  0.0099 units
   Bolus Amount MAE: 2.500 units (8 events)

 DETECTION BREAKDOWN:
   True Positives:   8 (correct bolus predictions)
   False Positives:  5 (false alarms)
   False Negatives:  4 (missed bolus events)
   True Negatives:  14 (correct no-bolus p